In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, losses, metrics, callbacks, datasets
from tensorflow.keras.utils import plot_model
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from joblib import dump, load
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
seed=42
tf.random.set_seed(seed)

C:\Users\india\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## using neural net on already labelled data

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [4]:
X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [5]:
X_train_full[0]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,
          0,   0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,
          1,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
          0,  36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,
          0,   3],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,
          0, 102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,
         10,   0],
       [  

In [6]:
X_train[0].shape

(28, 28)

In [5]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [8]:
history = model.fit(X_train, y_train, epochs=30,
                    validation_data=(X_valid, y_valid))

Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7239 - accuracy: 0.7643 - val_loss: 0.5211 - val_accuracy: 0.8216
Epoch 2/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4842 - accuracy: 0.8321 - val_loss: 0.4352 - val_accuracy: 0.8534
Epoch 3/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4388 - accuracy: 0.8455 - val_loss: 0.5341 - val_accuracy: 0.7984
Epoch 4/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4121 - accuracy: 0.8565 - val_loss: 0.3916 - val_accuracy: 0.8652
Epoch 5/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3937 - accuracy: 0.8619 - val_loss: 0.3747 - val_accuracy: 0.8692
Epoch 6/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3751 - accuracy: 0.8675 - val_loss: 0.3712 - val_accuracy: 0.8734
Epoch 7/30
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3631 - accuracy: 0.8714 - val_loss: 0.3626 - val_accuracy:

In [9]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3353 - accuracy: 0.8821


[0.33530178666114807, 0.882099986076355]

In [10]:
y_proba = model.predict(X_test)

313/313 [==============================] - 1s 2ms/step


In [11]:
np.argmax(y_proba, axis=-1)

array([9, 2, 1, ..., 8, 1, 5], dtype=int64)

In [12]:
y_test

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

#### we will be using semi supervised learning where i will take unlabelled data and label the data using the clustering on the subset of data then will use the supervised learning get the label on all the data. we will validate the accuracy of neural network by checking on already labeled untrained data

## Let's Use Clustering for label (K mean clustering)

In [13]:
## take only X_train dataset and take their subset to make label

X_train_subset= X_train[:40000]
X_train_subset_flatten= X_train_subset.reshape(X_train[:40000].shape[0],-1)
## using clustering on the this subset
kmeans = KMeans(init='k-means++', n_clusters=50, random_state=0).fit(X_train_subset_flatten)



In [14]:
label=kmeans.labels_  ## label of the subset data

In [15]:
## doing test train split for validation
X_train_full_new, X_test_new= X_train_subset[:35000],X_train_subset[35000:]
Y_train_full_new, Y_test_new= label[:35000], label[35000:]

In [16]:
X_valid_new, X_train_new = X_train_full_new[:5000], X_train_full_new[5000:]
Y_valid_new, Y_train_new = Y_train_full_new[:5000], Y_train_full_new[5000:]

In [17]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(50, activation="softmax")
])

In [18]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [19]:
history = model.fit(X_train_new, Y_train_new, epochs=30,
                    validation_data=(X_valid_new, Y_valid_new))

Epoch 1/30
938/938 [==============================] - 4s 4ms/step - loss: 2.2739 - accuracy: 0.3861 - val_loss: 1.3574 - val_accuracy: 0.5730
Epoch 2/30
938/938 [==============================] - 3s 4ms/step - loss: 1.1093 - accuracy: 0.6333 - val_loss: 0.9669 - val_accuracy: 0.6748
Epoch 3/30
938/938 [==============================] - 3s 4ms/step - loss: 0.8300 - accuracy: 0.7040 - val_loss: 0.7501 - val_accuracy: 0.7330
Epoch 4/30
938/938 [==============================] - 3s 3ms/step - loss: 0.6769 - accuracy: 0.7535 - val_loss: 0.6383 - val_accuracy: 0.7700
Epoch 5/30
938/938 [==============================] - 3s 3ms/step - loss: 0.5718 - accuracy: 0.7925 - val_loss: 0.5734 - val_accuracy: 0.7798
Epoch 6/30
938/938 [==============================] - 3s 3ms/step - loss: 0.4917 - accuracy: 0.8245 - val_loss: 0.5299 - val_accuracy: 0.7992
Epoch 7/30
938/938 [==============================] - 3s 3ms/step - loss: 0.4301 - accuracy: 0.8480 - val_loss: 0.4279 - val_accuracy: 0.8434
Epoch 

In [20]:
model.evaluate(X_test_new, Y_test_new)  

157/157 [==============================] - 0s 2ms/step - loss: 0.1953 - accuracy: 0.9194


[0.1952732354402542, 0.9193999767303467]

### acheived a accuracy of 91% by doing labeling on unlabeled data with 50 cluster

In [21]:
## acheive a accuracy of 91% with 

In [22]:
y_proba_new = model.predict(X_test_new)

157/157 [==============================] - 0s 2ms/step


In [23]:
np.argmax(y_proba_new, axis=-1)

array([47, 10, 25, ..., 39,  5, 39], dtype=int64)

In [24]:
Y_test_new

array([47, 10, 25, ..., 39,  5, 28])

In [25]:
## getting label on data which was not in our subset
label_rest= model.predict(X_train[40000:])

469/469 [==============================] - 1s 2ms/step


## experimenting with different value of cluster
### Taking cluster of 30

In [26]:

X_train_subset_30= X_train[:40000]
X_train_subset_flatten_30= X_train_subset_30.reshape(X_train[:40000].shape[0],-1)
## using clustering on the this subset
kmeans = KMeans(init='k-means++', n_clusters=30, random_state=0).fit(X_train_subset_flatten_30)



In [27]:
label_30=kmeans.labels_  ## label of the subset data with 30 cluster

In [28]:
## doing test train split for validation
X_train_full_new_30, X_test_new_30= X_train_subset_30[:35000],X_train_subset_30[35000:]
Y_train_full_new_30, Y_test_new_30= label_30[:35000], label_30[35000:]

In [29]:
X_valid_new_30, X_train_new_30 = X_train_full_new_30[:5000], X_train_full_new_30[5000:]
Y_valid_new_30, Y_train_new_30 = Y_train_full_new_30[:5000], Y_train_full_new_30[5000:]

In [30]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(30, activation="softmax")
])

In [31]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [32]:
history = model.fit(X_train_new_30, Y_train_new_30, epochs=30,
                    validation_data=(X_valid_new_30, Y_valid_new_30))

Epoch 1/30
938/938 [==============================] - 4s 4ms/step - loss: 1.6867 - accuracy: 0.5126 - val_loss: 0.9355 - val_accuracy: 0.6920
Epoch 2/30
938/938 [==============================] - 3s 3ms/step - loss: 0.7823 - accuracy: 0.7208 - val_loss: 0.6942 - val_accuracy: 0.7368
Epoch 3/30
938/938 [==============================] - 3s 3ms/step - loss: 0.5925 - accuracy: 0.7815 - val_loss: 0.5432 - val_accuracy: 0.7888
Epoch 4/30
938/938 [==============================] - 3s 4ms/step - loss: 0.4834 - accuracy: 0.8212 - val_loss: 0.4417 - val_accuracy: 0.8398
Epoch 5/30
938/938 [==============================] - 4s 4ms/step - loss: 0.4047 - accuracy: 0.8528 - val_loss: 0.3812 - val_accuracy: 0.8586
Epoch 6/30
938/938 [==============================] - 4s 4ms/step - loss: 0.3474 - accuracy: 0.8745 - val_loss: 0.3633 - val_accuracy: 0.8636
Epoch 7/30
938/938 [==============================] - 3s 4ms/step - loss: 0.3038 - accuracy: 0.8940 - val_loss: 0.3245 - val_accuracy: 0.8734
Epoch 

In [33]:
model.evaluate(X_test_new_30, Y_test_new_30)  

157/157 [==============================] - 0s 2ms/step - loss: 0.1531 - accuracy: 0.9362


[0.15311087667942047, 0.9362000226974487]

In [34]:
## getting label on data which was not in our subset
label_rest= model.predict(X_train[40000:])

469/469 [==============================] - 1s 2ms/step


## after taking a value of K=30 achieve a accuracy of 94%

## Taking K=10

In [35]:

X_train_subset_10= X_train[:40000]
X_train_subset_flatten_10= X_train_subset_10.reshape(X_train[:40000].shape[0],-1)
## using clustering on the this subset
kmeans = KMeans(init='k-means++', n_clusters=10, random_state=0).fit(X_train_subset_flatten_10)



In [36]:
label_10=kmeans.labels_  ## label of the subset data with 30 cluster

In [37]:
## doing test train split for validation
X_train_full_new_10, X_test_new_10= X_train_subset_10[:35000],X_train_subset_10[35000:]
Y_train_full_new_10, Y_test_new_10= label_10[:35000], label_10[35000:]

In [38]:
X_valid_new_10, X_train_new_10 = X_train_full_new_10[:5000], X_train_full_new_10[5000:]
Y_valid_new_10, Y_train_new_10 = Y_train_full_new_10[:5000], Y_train_full_new_10[5000:]

In [39]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [40]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [41]:
history = model.fit(X_train_new_10, Y_train_new_10, epochs=30,
                    validation_data=(X_valid_new_10, Y_valid_new_10))

Epoch 1/30
938/938 [==============================] - 4s 3ms/step - loss: 0.6654 - accuracy: 0.7635 - val_loss: 0.3808 - val_accuracy: 0.8578
Epoch 2/30
938/938 [==============================] - 3s 3ms/step - loss: 0.3300 - accuracy: 0.8727 - val_loss: 0.2711 - val_accuracy: 0.9004
Epoch 3/30
938/938 [==============================] - 3s 3ms/step - loss: 0.2492 - accuracy: 0.9070 - val_loss: 0.2246 - val_accuracy: 0.9164
Epoch 4/30
938/938 [==============================] - 3s 3ms/step - loss: 0.2036 - accuracy: 0.9215 - val_loss: 0.2025 - val_accuracy: 0.9230
Epoch 5/30
938/938 [==============================] - 3s 3ms/step - loss: 0.1740 - accuracy: 0.9358 - val_loss: 0.1596 - val_accuracy: 0.9402
Epoch 6/30
938/938 [==============================] - 3s 3ms/step - loss: 0.1541 - accuracy: 0.9434 - val_loss: 0.2547 - val_accuracy: 0.8970
Epoch 7/30
938/938 [==============================] - 3s 3ms/step - loss: 0.1361 - accuracy: 0.9503 - val_loss: 0.1920 - val_accuracy: 0.9194
Epoch 

In [42]:
model.evaluate(X_test_new_10, Y_test_new_10)  

157/157 [==============================] - 0s 2ms/step - loss: 0.0943 - accuracy: 0.9582


[0.0942617803812027, 0.9581999778747559]

In [43]:
## getting label on data which was not in our subset
label_rest= model.predict(X_train[40000:])

469/469 [==============================] - 1s 2ms/step


## Taking K=100

In [44]:

X_train_subset_100= X_train[:40000]
X_train_subset_flatten_100= X_train_subset_100.reshape(X_train[:40000].shape[0],-1)
## using clustering on the this subset
kmeans = KMeans(init='k-means++', n_clusters=100, random_state=0).fit(X_train_subset_flatten_100)



In [45]:
label_100=kmeans.labels_  ## label of the subset data with 100 cluster

In [46]:
## doing test train split for validation
X_train_full_new_100, X_test_new_100= X_train_subset_100[:35000],X_train_subset_100[35000:]
Y_train_full_new_100, Y_test_new_100= label_100[:35000], label_100[35000:]

In [47]:
X_valid_new_100, X_train_new_100 = X_train_full_new_100[:5000], X_train_full_new_100[5000:]
Y_valid_new_100, Y_train_new_100 = Y_train_full_new_100[:5000], Y_train_full_new_100[5000:]

In [48]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(100, activation="softmax")
])

In [49]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [50]:
history = model.fit(X_train_new_100, Y_train_new_100, epochs=30,
                    validation_data=(X_valid_new_100, Y_valid_new_100))

Epoch 1/30
938/938 [==============================] - 4s 4ms/step - loss: 3.4466 - accuracy: 0.1956 - val_loss: 2.3373 - val_accuracy: 0.3426
Epoch 2/30
938/938 [==============================] - 3s 3ms/step - loss: 1.8359 - accuracy: 0.4640 - val_loss: 1.5523 - val_accuracy: 0.4966
Epoch 3/30
938/938 [==============================] - 3s 3ms/step - loss: 1.3162 - accuracy: 0.5813 - val_loss: 1.2295 - val_accuracy: 0.5842
Epoch 4/30
938/938 [==============================] - 3s 3ms/step - loss: 1.0646 - accuracy: 0.6488 - val_loss: 0.9912 - val_accuracy: 0.6666
Epoch 5/30
938/938 [==============================] - 3s 4ms/step - loss: 0.9075 - accuracy: 0.6904 - val_loss: 0.8768 - val_accuracy: 0.6906
Epoch 6/30
938/938 [==============================] - 4s 4ms/step - loss: 0.7858 - accuracy: 0.7287 - val_loss: 0.7790 - val_accuracy: 0.7272
Epoch 7/30
938/938 [==============================] - 4s 4ms/step - loss: 0.6890 - accuracy: 0.7592 - val_loss: 0.7280 - val_accuracy: 0.7334
Epoch 

In [51]:
model.evaluate(X_test_new_100, Y_test_new_100)  

157/157 [==============================] - 0s 3ms/step - loss: 0.2785 - accuracy: 0.8920


[0.27845636010169983, 0.8920000195503235]

### with cluster of 100 we will get the accuarcy of 88.86%

In [52]:
## getting label on data which was not in our subset
label_rest= model.predict(X_train[40000:])

469/469 [==============================] - 1s 2ms/step


## conclusion

#### if we decrease the cluster size accuracy will increase but no of labels will decrease which will not be a good classification as it will classify two different object in same label, but if we incrase the label I.e. the cluster size then accuracy of classification in different label will decrease 